<a href="https://colab.research.google.com/github/vilsonrodrigues/Projeto_Orientacao_Academica/blob/master/Projeto_Orientacao_Academica_Gerador_de_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# <center><font color='blue'>Projeto de Orientação Acadêmica</font>




<img src='https://ufrn.br/resources/documentos/identidadevisual/logotipo/logotipo_flat.png'>

## Parte I - Apresentação do Problema
### [1. Apresentação do Problema](#apresentacao)
## Parte II - Preparativos Iniciais
### [1. Bibliotecas](#bibliotecas)
### [2. Classe](#classe)
### [3. Funções](#funcoes)
### [4. Leitura da Base](#leiturabase)
## Parte III - Desenvolvimento
### [1. Entendimento de Dados](#enten_dados)
### [2. Métricas de Avaliação](#metricas)
### [3. Modelagem](#modelagem)
### [4. Avaliação de Modelo](#avaliacao)
### [5. Conclusão](#conclusao)

# Parte I - Apresentação do Problema

Nossa meta é prever através de Machine Learning as probabilidades de um estudante ter sucesso em uma disciplina dado o histórico escolar na Universidade.

Nossos dados foram transformados para ficarem de uma forma génerica, assim, não dependemos de saber qual foi a matéria cursada anteriormente.

Com essa plataforma, os professores que orientam alunos, terão mais segurança e uma comprovação com base no Machine Learning que estudantes que tiveram um histórico semelhante e seguiram esse caminho tiveram mais êxito do que seguir por este outro.

Os dados foram coletados do período de 2015.1 até 2019.1.

**Esse notebook foca totalmente em gerar os modelos para cada discipina do ciclo básico da ECT-UFRN.**

### Fonte dos Dados
Dados foram extraídos inicialmetne do portal de Dados Abertos da UFRN, depois foram tratados em outro [notebook](https://).
### Referência
Hands On: Machine Learning with Scikit-Learning e Tensor Flow



# Parte II - Preparativos Iniciais

## 1. Bibliotecas
<a id='bibliotecas'></a>
#### [1.1 Instalações](#instalacao)
#### [1.2 Importações](#importacao)


### 1.1. Instalação
<a id='instalacao'></a>

In [0]:
!pip install pandas -U
!pip install scikit-learn -U
!pip install numpy -U
!pip install joblib==0.14.1
!conda install xgboost -c conda-forge --yes

### 1.2. Importação
<a id='importacao'></a>

In [0]:
import pandas as pd
import numpy as np

#Persistência em disco
import joblib
import pickle

#Visualização
import seaborn as sns
import matplotlib.pyplot as plt

#Transformadores e Modelador
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

#Selecao de modelo
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2

#Metricas
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

#Classificadores
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#modo não-perturbe
import warnings
warnings.filterwarnings("ignore")

##2. Classe
<a id='classe'></a>


Classe para discretizar dados

In [0]:
class NumericalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, model=0):
     self.model = model
  def fit(self, X, y = None ):
    return self 

  def create_dummies(self, df, column_name, drop_first_col):
    #Cria colunas fictícias apartir de uma
    dummies = pd.get_dummies(df[column_name],prefix=column_name, drop_first=drop_first_col)
    return dummies

  #Manipulando a coluna "media"
  def process_media(self, df):
    #divide a coluna de média em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["media_categorias"] = pd.cut(df["media"],
                                    cut_points_notas,
                                    labels=label_names_notas)
    return self.create_dummies(df,"media_categorias",False)
   
  #Manipulando a coluna "min"
  def process_min(self, df):
    #divide a coluna min em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["min_categorias"] = pd.cut(df["min"],
                                  cut_points_notas,
                                  labels=label_names_notas)
    return self.create_dummies(df,"min_categorias",False) 

  #Manipulando a coluna "max"
  def process_max(self, df):
    #divide a coluna max em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["max_categorias"] = pd.cut(df["max"],
                                  cut_points_notas,
                                  labels=label_names_notas)
    return self.create_dummies(df,"max_categorias",False) 

  #Manipulando a coluna "carga_total"
  def process_carga_total(self, df):
    #divide a coluna carga_total em um intervalo de valores 
    cut_points_carga = [0,100,200,300,400,600]
    label_names_carga = ["muito baixo","baixo","media","alto","muito alto"] 
    df["carga_total_categorias"] = pd.cut(df["carga_total"],
                                          cut_points_carga,
                                          labels=label_names_carga)
    return self.create_dummies(df,"carga_total_categorias",False) 

  #Manipulando a coluna "carga_total_atual"
  def process_carga_total_atual(self, df):
    #divide a coluna carga_total_atual em um intervalo de valores 
    cut_points_carga = [0,100,200,300,400,600]
    label_names_carga = ["muito baixo","baixo","media","alto","muito alto"] 
    df["carga_total_atual_categorias"] = pd.cut(df["carga_total_atual"],
                                                cut_points_carga,
                                                labels=label_names_carga)
    return self.create_dummies(df,"carga_total_atual_categorias",False) 
  
  #Criando a coluna de Taxa de sucesso
  def tax_suc(self, df):
    df["tax_suc"] = df["quantAprovado"] / df["QuantDisciCursadas"]
    return pd.DataFrame(df["tax_suc"],columns=["tax_suc"])

  #Concatenando colunas
  def addcol(sefl, df):
    QuantDisciCursadas = 	df["QuantDisciCursadas"]
    quantAprovado      = 	df["quantAprovado"]
    quantReprovado     =  df["quantReprovado"]
    QuantDisciAtual    =  df["QuantDisciAtual"]
    AnosMatriculado    =  df["AnosMatriculado"]
    return pd.concat([QuantDisciCursadas, quantAprovado, quantReprovado, QuantDisciAtual,AnosMatriculado ],axis=1)


  #Método transformador
  def transform(self, X , y = None ):
    df = X.copy()
    media = self.process_media(df)  
    min1 = self.process_min(df)
    max1 = self.process_max(df)
    carga_total = self.process_carga_total(df)
    carga_total_atual = self.process_carga_total_atual(df)
    tax_suc = self.tax_suc(df)
    ad = self.addcol(df)
      
    return pd.concat([media,min1,max1,carga_total,carga_total_atual,tax_suc,ad],axis=1)

##3. Funções
<a id='funcoes'></a>

Função que usa GridSearch para avaliar o melhor modelo disponível, retornar estatísticas dos resultados e persistir em disco os modelos gerados

In [0]:
def avaliar_modelos(df, hyperparameters):

  #Dicionários
  model_param_disciplinas = {}
  best_acc_disciplinas = {}
  scoring_disciplinas = {}
  #feature_importances_disciplinas = {}

  #Seleciona o codigo de cada disciplina no dataframe
  lista_disciplinas = df["disciplina"].unique()  
  #Para fins de teste, vou passar apenas 2 disciplinas pra gerar modelo
  #lista_disciplinas = ['ECT2411', 'ECT2304']

  #Verifica se esta vazio
  if lista_disciplinas is None:
	  return None

  #Convertendo categorias do target de string para numerico (0,1)
  df["situacao_categoria"] = pd.Categorical(df["situacao"]).codes

  #Pipeline para transformar em categoria e verificar melhores parametros
  pipeline_transform = Pipeline(steps = [('num_pipeline', NumericalTransformer()),
                                         ('fs',SelectKBest(chi2)),
                                         ('sc',StandardScaler()),
                                         ('clf',XGBClassifier())
                               ])

  #Grid para testar todos os parâmetros                               
  grid_search = GridSearchCV(
                          estimator = pipeline_transform, 
                          param_grid = hyperparameters,
                          cv= 5,
                          scoring = {'Accuracy': make_scorer(accuracy_score)},
                          #scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score), "F1":"f1", "Recall":"recall", "Precision":"precision"},
                          return_train_score=True,
                          n_jobs=-1,#indica o numero de processos em paralelo, -1 significa usar todos
                          refit='Accuracy')


  #itera sobre a lista de disciplinas
  for disciplina in lista_disciplinas:
    data_iter = df.loc[df["disciplina"].str.contains(disciplina),:]
    
    #Treinando Grid
    best_model = grid_search.fit(data_iter.drop(columns = ["disciplina","situacao_categoria"]), data_iter["situacao_categoria"])
 
    #Extraindo informações do treino
    param_modelo = best_model.best_params_
    model_param_disciplinas[disciplina] = param_modelo['clf']
    best_acc_disciplinas[disciplina] = best_model.best_score_

    #Resultado de metricas
    result = pd.DataFrame(best_model.cv_results_)
    result = result[['mean_train_Accuracy', 'std_train_Accuracy','mean_test_Accuracy', 'std_test_Accuracy','rank_test_Accuracy']].copy()
    result["std_ratio"] = result.std_test_Accuracy/result.std_train_Accuracy
    result = result.sort_values(by="rank_test_Accuracy",ascending=True)    
    scoring_disciplinas[disciplina] = result.iloc[0,:]

    #Características mais importantes
    #feature_importances_disciplinas[disciplina] = grid.best_estimator_.feature_importances_
    
    #Salvando modelo
    joblib.dump(best_model, 'modelo_' + disciplina + '.pkl')


  #Transformando o dicionário para Dataframe
  df_best_acc = pd.DataFrame.from_dict(best_acc_disciplinas, orient="index")
  df_best_acc.columns = ["accuracy"]
  df_scor_disc = pd.DataFrame.from_dict(scoring_disciplinas, orient="index")

  #Concatenando DataFrames
  df_relatorio = pd.concat([df_best_acc, df_scor_disc], axis=1, sort=False)

  return (df_relatorio,model_param_disciplinas)

##4. Leitura da Base
<a id='leiturabase'></a>

In [47]:
df = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/dados%20tratados%20ufrn/disciplinasECT201520191.csv")
df.drop(columns=["Unnamed: 0","matricula","siape"], inplace=True)
df.head()

,AnosMatriculado,QuantDisciAtual,QuantDisciCursadas,anoSemestre,anoSemestreAtual,carga_total,carga_total_atual,disciplina,max,media,mediasAprovadas,mediasReprovadas,min,quantAprovado,quantReprovado,situacao,vezesReprovado
0,6,5,4,2015.1,2015.2,210,330,ECT2411,6.1,3.950000,5.900000,2.0,0.0,2,2,REPROVADO,0.0
1,6,5,4,2015.1,2015.2,210,330,ECT2304,6.1,3.950000,5.900000,2.0,0.0,2,2,REPROVADO,0.0
2,6,5,4,2015.1,2015.2,210,330,ECT2303,6.1,3.950000,5.900000,2.0,0.0,2,2,REPROVADO,0.0
3,6,8,6,2015.1,2015.2,360,480,ECT2412,7.2,4.033333,6.266667,1.8,0.0,3,3,APROVADO,0.0
4,6,8,6,2015.1,2015.2,360,480,ECT2414,7.2,4.033333,6.266667,1.8,0.0,3,3,REPROVADO,0.0


#Parte III - Desenvolvimento
<a id='enten_dados'></a>


## 1. Entendimento de Dados
<a id='enten_dados'></a>
### [1.1 Descrição de Dados](#descricao)
### [1.2 Qualidade de Dados](#qualidade)

###1.1 Descrição de Dados
<a id='descricao'></a>


#### Número de Atributos (colunas): 17 
#### Número de Registros (linhas): 53.663 registros 

Base contêm 53.663 dados sem nenhum valor não-nulo. 

#### <center>Tipos dos Atributos:
| Inteiros | Floats(Ponto Flutuante) | Booleano (Binários) | String (Texto) |
| --- | --- | --- | --- |
| 7 | 8 | 1 | 1 |






In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53663 entries, 0 to 53662
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AnosMatriculado     53663 non-null  int64  
 1   QuantDisciAtual     53663 non-null  int64  
 2   QuantDisciCursadas  53663 non-null  int64  
 3   anoSemestre         53663 non-null  float64
 4   anoSemestreAtual    53663 non-null  float64
 5   carga_total         53663 non-null  int64  
 6   carga_total_atual   53663 non-null  int64  
 7   disciplina          53663 non-null  object 
 8   max                 53663 non-null  float64
 9   media               53663 non-null  float64
 10  mediasAprovadas     53663 non-null  float64
 11  mediasReprovadas    53663 non-null  float64
 12  min                 53663 non-null  float64
 13  quantAprovado       53663 non-null  int64  
 14  quantReprovado      53663 non-null  int64  
 15  situacao            53663 non-null  object 
 16  veze

In [0]:
df.select_dtypes(include=['int64','float64']).describe()

In [0]:
profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})

###1.2 Qualidade dos dados
<a id='qualidade'></a>

Plotando correlação entre os dados númericos

In [0]:
#corr = df.select_dtypes(include=['int64','float64']).corr()
corr = df.corr()
corr = corr.style.background_gradient(cmap='Blues')
corr

,AnosMatriculado,QuantDisciAtual,QuantDisciCursadas,anoSemestre,anoSemestreAtual,carga_total,carga_total_atual,max,media,mediasAprovadas,mediasReprovadas,min,quantAprovado,quantReprovado,vezesReprovado
AnosMatriculado,1.000000,-0.244676,-0.219483,-0.018912,0.064563,-0.086665,-0.130018,-0.184681,-0.243871,-0.190578,0.078361,-0.205659,-0.268878,0.150141,0.297610
QuantDisciAtual,-0.244676,1.000000,0.419941,-0.123166,-0.132986,0.408583,0.930837,0.262090,0.334584,0.160091,-0.047112,0.275943,0.461997,-0.219400,-0.159623
QuantDisciCursadas,-0.219483,0.419941,1.000000,-0.061528,-0.054679,0.928605,0.412855,0.301823,0.205117,0.139712,0.058751,0.068883,0.637323,0.076397,-0.051482
anoSemestre,-0.018912,-0.123166,-0.061528,1.000000,0.941378,-0.087243,-0.108739,0.030676,0.017577,0.025671,0.006055,0.008108,-0.025949,-0.021862,0.154607
anoSemestreAtual,0.064563,-0.132986,-0.054679,0.941378,1.000000,-0.083201,-0.118071,0.028583,0.012132,0.020410,0.007903,0.002094,-0.024091,-0.018096,0.165494
carga_total,-0.086665,0.408583,0.928605,-0.087243,-0.083201,1.000000,0.419023,0.237054,0.130630,0.076360,0.084471,0.015747,0.549117,0.126198,-0.003368
carga_total_atual,-0.130018,0.930837,0.412855,-0.108739,-0.118071,0.419023,1.000000,0.253294,0.310090,0.148443,-0.019729,0.250245,0.437720,-0.194372,-0.090831
max,-0.184681,0.262090,0.301823,0.030676,0.028583,0.237054,0.253294,1.000000,0.753144,0.849063,-0.242554,0.506297,0.596593,-0.499982,-0.156451
media,-0.243871,0.334584,0.205117,0.017577,0.012132,0.130630,0.310090,0.753144,1.000000,0.677409,-0.279229,0.853552,0.789567,-0.836807,-0.271177
mediasAprovadas,-0.190578,0.160091,0.139712,0.025671,0.020410,0.076360,0.148443,0.849063,0.677409,1.000000,-0.256957,0.469960,0.387764,-0.375845,-0.119652


Falar um pouco da correlação

##2. Métricas de Avaliação
<a id='metricas'></a>

Ainda não coloquei

##3. Modelagem
<a id='modelagem'></a>

Dicionário com hiper-parâmetros dos algoritmos preditivos

In [0]:
hyperparameters = [
                  {"clf":[RandomForestClassifier()],
                  "clf__n_estimators": [100],
                  "clf__criterion": ["entropy"],
                  "clf__max_leaf_nodes": [64],
                  "clf__random_state": [42],
                  "fs__score_func":[chi2],
                  "fs__k":[4,9,15,30]
                  },
              
                  {"clf":[KNeighborsClassifier()],
                  "clf__n_neighbors":[5,9,11],
                  "fs__score_func":[chi2],
                  "fs__k":[4,9,15,30]                 
                  },
              
                  {"clf":[SVC()],
                  "clf__kernel":["sigmoid",'rbf'],
                  "clf__degree":[3,4],
                  "clf__gamma":[0.1,0.5,1],
                  "clf__C":[0.001,1,2],
                  "fs__score_func":[chi2],
                  "fs__k":[4,9,15,25,31] 
                  },
              
                  {"clf":[GaussianNB()]
                  },
              
                  {"clf":[MLPClassifier()],
                  "clf__hidden_layer_sizes": [(64,),(128,)],
                  "clf__activation": ["logistic"],
                  "clf__solver": ["sgd"],
                  "clf__max_iter": [500],
                  "clf__early_stopping":[True],
                  "clf__n_iter_no_change":[20],
                  "clf__validation_fraction":[0.20], 
                  },
              
                  {"clf":[XGBClassifier()],
                  "clf__n_estimators": [50,100],
                  "clf__max_depth": [4,6],
                  "clf__learning_rate": [0.001, 0.01,0.1],
                  "clf__random_state": [42],
                  "clf__subsample": [1.0],
                  "clf__colsample_bytree": [1.0],
                  "fs__score_func":[chi2],
                  "fs__k":[5,8,15,25,31]
                  }
  ]

##4. Avaliação dos Modelos
<a id='avaliacao'></a>

### [4.1 Chamada da Função](#chamada)
### [4.2 Teste de Modelo](#teste)

###4.1 Chamada da Função
<a id='chamada'></a>

A fins de testes só passei 2 disciplinas como teste, mas faz para todas

In [0]:
(df_relatorio, model_param_disciplinas) = avaliar_modelos(df, hyperparameters)

In [43]:
df_relatorio.head()

,accuracy,mean_train_Accuracy,std_train_Accuracy,mean_test_Accuracy,std_test_Accuracy,rank_test_Accuracy,std_ratio
ECT2411,0.697063,0.719136,0.003203,0.697063,0.010034,1.0,3.132646
ECT2304,0.709524,0.730195,0.008443,0.696104,0.042605,91.0,5.046221


In [44]:
model_param_disciplinas

{'ECT2304': SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma=0.1, kernel='sigmoid',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False),
 'ECT2411': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='entropy', max_depth=None, max_features='auto',
                        max_leaf_nodes=64, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=42, verbose=0,
                        warm_start=False)}

###4.2 Teste de Modelo
<a id='teste'></a>

Importando Modelo

In [0]:
modelo_teste = joblib.load("modelo_ECT2304.pkl")

Aplicando a transformação numérica no teste

In [0]:
num_transf = NumericalTransformer()
df_teste = model.transform(df)

In [51]:
df_teste.head(2)

,media_categorias_muito baixo,media_categorias_baixo,media_categorias_media,media_categorias_alto,media_categorias_muito alto,min_categorias_muito baixo,min_categorias_baixo,min_categorias_media,min_categorias_alto,min_categorias_muito alto,max_categorias_muito baixo,max_categorias_baixo,max_categorias_media,max_categorias_alto,max_categorias_muito alto,carga_total_categorias_muito baixo,carga_total_categorias_baixo,carga_total_categorias_media,carga_total_categorias_alto,carga_total_categorias_muito alto,carga_total_atual_categorias_muito baixo,carga_total_atual_categorias_baixo,carga_total_atual_categorias_media,carga_total_atual_categorias_alto,carga_total_atual_categorias_muito alto,tax_suc,QuantDisciCursadas,quantAprovado,quantReprovado,QuantDisciAtual,AnosMatriculado
0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0.5,4,2,2,5,6
1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0.5,4,2,2,5,6


Realizando teste de um modelo, prevendo probabilidade de ser aprovado

In [0]:
df_teste.iloc[42,:]

In [55]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53663 entries, 0 to 53662
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   media_categorias_muito baixo              53663 non-null  uint8  
 1   media_categorias_baixo                    53663 non-null  uint8  
 2   media_categorias_media                    53663 non-null  uint8  
 3   media_categorias_alto                     53663 non-null  uint8  
 4   media_categorias_muito alto               53663 non-null  uint8  
 5   min_categorias_muito baixo                53663 non-null  uint8  
 6   min_categorias_baixo                      53663 non-null  uint8  
 7   min_categorias_media                      53663 non-null  uint8  
 8   min_categorias_alto                       53663 non-null  uint8  
 9   min_categorias_muito alto                 53663 non-null  uint8  
 10  max_categorias_muito baixo        

In [0]:
modelo_teste.predict(df_teste.iloc[42,:])

##5. Conclusão
<a id='conclusao'></a>